## This Notebook demonstrates how to reduce the bias during "In-processing" stage using AI 360 Fairness toolkit

### In-processing algorithm
A bias mitigation algorithm that is applied to a model during its training.

### Insert your credentials as credentials in the below cell
Click on dropdown from Pipeline_LabelEncoder-0.1.zip under Data tab and select 'Credentials'

In [4]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-d1dc205c-4852-439d-aaf7-025bfd1d1b3c',
    'IBM_API_KEY_ID': 'dX403mHGXVg4BfGcsmJljAoDLJazlII_c0b_3F-3ZX4K',
    'ENDPOINT': 'https://s3.private.us.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'aiethics-donotdelete-pr-8eprek5rk4yce3',
    'FILE': 'Pipeline_LabelEncoder-0.1.zip'
}


In [5]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [6]:
import os
os.getcwd()

'/home/wsuser/work'

In [7]:
cos.download_file(Bucket=credentials['BUCKET'],Key='Pipeline_LabelEncoder-0.1.zip',Filename='/home/wsuser/work/Pipeline_LabelEncoder-0.1.zip')

In [8]:
!ls

Pipeline_LabelEncoder-0.1.zip


In [9]:
!pip install Pipeline_LabelEncoder-0.1.zip
!pip install aif360

Processing ./Pipeline_LabelEncoder-0.1.zip
  Created wheel for Pipeline-LabelEncoder: filename=Pipeline_LabelEncoder-0.1-py3-none-any.whl size=2062 sha256=bbe6ef22a7571bfadd6ca44928b0aba33ec6f37eea3edcc551e47a4d5c371b5a
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/6b/4b/1e/43f3c8b97ffade4539a329b9eaa5755e4df16a248960947534
Successfully built Pipeline-LabelEncoder
     |████████████████████████████████| 175 kB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 371 kB 59.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 296 kB 53.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 74.8 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 26.3 MB 25.8 MB/s eta 0:00:01
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl 

In [10]:
import tensorflow as tf
tf.__version__

'2.4.3'

In [11]:
!pip install 'tensorflow>=1.13.1'

In [13]:
%matplotlib inline
# Load all necessary packages
import pandas as pd
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

### Insert the data as Pandas Dataframe and change the name from df_data_ to df

In [14]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_ef3d35c2a4224ff082f537a81071807f = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_ef3d35c2a4224ff082f537a81071807f = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_ef3d35c2a4224ff082f537a81071807f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='dX403mHGXVg4BfGcsmJljAoDLJazlII_c0b_3F-3ZX4K',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_ef3d35c2a4224ff082f537a81071807f)

body = client_ef3d35c2a4224ff082f537a81071807f.get_object(Bucket='aiethics-donotdelete-pr-8eprek5rk4yce3',Key='fraud_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Gender,Married,Education,Fraud_risk
0,Male,No,1,Risk
1,Male,Yes,1,Safe
2,Male,Yes,1,Safe
3,Male,Yes,0,Safe
4,Male,No,1,Risk


In [15]:
df.describe(include = 'all')

,Gender,Married,Education,Fraud_risk
count,921,921,921.000000,921
unique,2,2,NaN,2
top,Male,No,NaN,Safe
freq,703,501,NaN,562
mean,NaN,NaN,0.730727,NaN
std,NaN,NaN,0.443823,NaN
min,NaN,NaN,0.000000,NaN
25%,NaN,NaN,0.000000,NaN
50%,NaN,NaN,1.000000,NaN
75%,NaN,NaN,1.000000,NaN


In [16]:
privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]
favorable_label = 1 
unfavorable_label = 0

In [17]:
from sklearn import preprocessing
categorical_column = ['Gender', 'Married', 'Fraud_risk']

data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_column:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)
        

data_encoded.head()

Feature Gender
mapping {'Female': 0, 'Male': 1}
Feature Married
mapping {'No': 0, 'Yes': 1}
Feature Fraud_risk
mapping {'Risk': 0, 'Safe': 1}


,Gender,Married,Education,Fraud_risk
0,1,0,1,0
1,1,1,1,1
2,1,1,1,1
3,1,1,0,1
4,1,0,1,0


In [18]:
from Pipeline_LabelEncoder.sklearn_label_encoder import PipelineLabelEncoder
preprocessed_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).fit_transform(data_encoded)
print('-------------------------')
#print('validation data encoding')
#validation_enc_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).transform(validation_input_data)

Inside fit transform
Feature Gender
mapping {0: 0, 1: 1}
Feature Married
mapping {0: 0, 1: 1}
Feature Fraud_risk
mapping {0: 0, 1: 1}
-------------------------


In [19]:
#Create binary label dataset that can be used by bias mitigation algorithms
fraud_dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=preprocessed_data,
                                label_names=['Fraud_risk'],
                                protected_attribute_names=['Gender', 'Married'],
                                unprivileged_protected_attributes=unprivileged_groups)

In [20]:
display(Markdown("#### Training Data Details"))
print("shape of the training dataset", fraud_dataset.features.shape)
print("Training data favorable label", fraud_dataset.favorable_label)
print("Training data unfavorable label", fraud_dataset.unfavorable_label)
print("Training data protected attribute", fraud_dataset.protected_attribute_names)
print("Training data privileged protected attribute (1:Male and 0:Female)", 
      fraud_dataset.privileged_protected_attributes)
print("Training data unprivileged protected attribute (1:Male and 0:Female)",
      fraud_dataset.unprivileged_protected_attributes)

#### Training Data Details

shape of the training dataset (921, 3)
Training data favorable label 1.0
Training data unfavorable label 0.0
Training data protected attribute ['Gender', 'Married']
Training data privileged protected attribute (1:Male and 0:Female) [array([1.]), array([1.])]
Training data unprivileged protected attribute (1:Male and 0:Female) [array([0.]), array([0.])]


In [21]:
fraud_dataset_train, fraud_dataset_test = fraud_dataset.split([0.9], shuffle=True)

In [22]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(fraud_dataset_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(fraud_dataset_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.371556
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.374822


In [23]:
min_max_scaler = MaxAbsScaler()
fraud_dataset_train.features = min_max_scaler.fit_transform(fraud_dataset_train.features)
fraud_dataset_test.features = min_max_scaler.transform(fraud_dataset_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(fraud_dataset_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(fraud_dataset_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.371556
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.374822


### Build plan classifier without debiasing

In [34]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
# sess = tf.Session()
sess = tf.compat.v1.Session()

plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [37]:
tf.compat.v1.disable_eager_execution()
plain_model.fit(fraud_dataset_train)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch 0; iter: 0; batch classifier loss: 0.666113
epoch 1; iter: 0; batch classifier loss: 0.645210
epoch 2; iter: 0; batch classifier loss: 0.600706
epoch 3; iter: 0; batch classifier loss: 0.576786
epoch 4; iter: 0; batch classifier loss: 0.552515
epoch 5; iter: 0; batch classifier loss: 0.519860
epoch 6; iter: 0; batch classifier loss: 0.523782
epoch 7; iter: 0; batch classifier loss: 0.535875
epoch 8; iter: 0; batch classifier loss: 0.498684
epoch 9; iter: 0; batch classifier loss: 0.493991
epoch 10; iter: 0; batch classifier loss: 0.470357
epoch 11; iter: 0; batch classifier loss: 0.470270
epoch 12; iter: 0; batch classifier loss: 0.446249
epoch 13; iter: 0; batch classifier loss: 0.433933
epoch 14; iter: 0; batch classifier loss: 0.418300
epoch 15; iter: 0; batch classifier loss: 0.410631
epoch 16; iter: 0; batch classifier loss: 0.400611
epoch 17; iter: 0; batch cla

### Apply the plain model to test data

In [38]:
dataset_nodebiasing_train = plain_model.predict(fraud_dataset_train)
dataset_nodebiasing_test = plain_model.predict(fraud_dataset_test)

### Metrics for the dataset from plain model (without debiasing)

In [39]:
display(Markdown("#### Model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(fraud_dataset_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.513849
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.507112


#### Model - without debiasing - classification metrics

Test set: Classification accuracy = 0.827957
Test set: Balanced classification accuracy = 0.806452
Test set: Disparate impact = 0.341644
Test set: Equal opportunity difference = -0.176623
Test set: Average odds difference = -0.298838
Test set: Theil_index = 0.119251


### Apply in-processing algorithm based on adversarial learning

In [42]:
sess.close()
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()

In [43]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [44]:
debiased_model.fit(fraud_dataset_train)

epoch 0; iter: 0; batch classifier loss: 0.720624; batch adversarial loss: 0.825836
epoch 1; iter: 0; batch classifier loss: 0.681766; batch adversarial loss: 0.793567
epoch 2; iter: 0; batch classifier loss: 0.698799; batch adversarial loss: 0.831468
epoch 3; iter: 0; batch classifier loss: 0.643142; batch adversarial loss: 0.803706
epoch 4; iter: 0; batch classifier loss: 0.639541; batch adversarial loss: 0.827873
epoch 5; iter: 0; batch classifier loss: 0.632800; batch adversarial loss: 0.815942
epoch 6; iter: 0; batch classifier loss: 0.581710; batch adversarial loss: 0.859766
epoch 7; iter: 0; batch classifier loss: 0.603391; batch adversarial loss: 0.819742
epoch 8; iter: 0; batch classifier loss: 0.557663; batch adversarial loss: 0.853227
epoch 9; iter: 0; batch classifier loss: 0.574924; batch adversarial loss: 0.841624
epoch 10; iter: 0; batch classifier loss: 0.512254; batch adversarial loss: 0.847106
epoch 11; iter: 0; batch classifier loss: 0.518143; batch adversarial loss:

### Apply the plain model to test data

In [45]:
dataset_debiasing_train = debiased_model.predict(fraud_dataset_train)
dataset_debiasing_test = debiased_model.predict(fraud_dataset_test)

In [46]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(fraud_dataset_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.513849
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.507112


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.222623
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.265292


#### Model - without debiasing - classification metrics

Test set: Classification accuracy = 0.827957
Test set: Balanced classification accuracy = 0.806452
Test set: Disparate impact = 0.341644
Test set: Equal opportunity difference = -0.176623
Test set: Average odds difference = -0.298838
Test set: Theil_index = 0.119251


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.817204
Test set: Balanced classification accuracy = 0.846774
Test set: Disparate impact = 0.543452
Test set: Equal opportunity difference = -0.049351
Test set: Average odds difference = -0.009324
Test set: Theil_index = 0.185230


## We have observed how to use AI 360 fairness toolkit to eliminate the bias during preprocessing & inprocessing stages of model building & development.